## Ex 2

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
import seaborn as sns

np.random.seed(42)

### 2.1.1

In [ ]:
X_train, _ = make_blobs(
    n_samples=500,
    n_features=2,
    centers=[[0, 0]],
    cluster_std=1.0,
    random_state=42
)

plt.scatter(X_train[:, 0], X_train[:, 1], alpha=0.6, s=25, c='blue')
plt.title('Training Data: Standard Normal Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.show()

### 2.1.2

In [ ]:
n_projections = 5
n_features = 2

mean = np.zeros(n_features)
cov = np.eye(n_features)

projection_vectors = np.random.multivariate_normal(
    mean=mean,
    cov=cov,
    size=n_projections
)

# Normalize to unit length
projection_vectors = projection_vectors / np.linalg.norm(projection_vectors, axis=1, keepdims=True)

print(n_projections)
print(projection_vectors)

# Visualize projection vectors
plt.figure(figsize=(8, 8))
plt.scatter(X_train[:, 0], X_train[:, 1], alpha=0.3, s=20, c='lightblue', label='Training data')

# Draw projection vectors as arrows from origin
colors = ['red', 'green', 'orange', 'purple', 'brown']
for i, (vec, color) in enumerate(zip(projection_vectors, colors)):
    plt.arrow(0, 0, vec[0]*2, vec[1]*2, 
             head_width=0.15, head_length=0.1, 
             fc=color, ec=color, linewidth=2, 
             label=f'Projection {i+1}')

plt.title('Training Data with Projection Vectors', fontsize=14, fontweight='bold')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)
plt.legend(loc='upper right')
plt.axis('equal')
plt.xlim(-4, 4)
plt.ylim(-4, 4)
plt.tight_layout()
plt.show()

In [ ]:
def build_histograms(X, projection_vectors, n_bins=10, range_margin=1.5):
    histograms = []
    probabilities = []
    
    for i, vec in enumerate(projection_vectors):
        # Project data onto this vector
        projections = X @ vec
        
        # Compute range for histogram
        data_min, data_max = projections.min(), projections.max()
        data_range = data_max - data_min
        hist_range = (data_min - range_margin * data_range, 
                     data_max + range_margin * data_range)
        
        # Build histogram
        counts, bin_edges = np.histogram(projections, bins=n_bins, range=hist_range)
        
        # Convert counts to probabilities
        # Add small epsilon to avoid division by zero
        probs = (counts + 1e-10) / (counts.sum() + 1e-10 * n_bins)
        
        histograms.append((counts, bin_edges))
        probabilities.append(probs)
        
    return histograms, probabilities

n_bins = 10
histograms, probabilities = build_histograms(X_train, projection_vectors, n_bins=n_bins)

print(probabilities[0])
print(f"Sum of probabilities: {probabilities[0].sum()}")

In [ ]:
# Visualize histograms for each projection
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

colors_hist = ['red', 'green', 'orange', 'purple', 'brown']

for i, (vec, (counts, bin_edges), probs, color) in enumerate(zip(
    projection_vectors, histograms, probabilities, colors_hist)):
    
    ax = axes[i]
    
    # Plot histogram
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    ax.bar(bin_centers, probs, width=(bin_edges[1]-bin_edges[0])*0.9, 
           color=color, alpha=0.7, edgecolor='black')
    
    ax.set_title(f'Projection {i+1}: [{vec[0]:.3f}, {vec[1]:.3f}]', 
                fontsize=12, fontweight='bold')
    ax.set_xlabel('Projected Value')
    ax.set_ylabel('Probability')
    ax.grid(True, alpha=0.3, axis='y')

plt.suptitle(f'Histograms for Each Projection ({n_bins} bins)', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
def compute_anomaly_scores(X_test, projection_vectors, histograms, probabilities):
    n_samples = X_test.shape[0]
    n_projections = len(projection_vectors)
    
    sample_probabilities = np.zeros((n_samples, n_projections))
    
    for i, vec in enumerate(projection_vectors):
        projections = X_test @ vec
        
        _, bin_edges = histograms[i]
        probs = probabilities[i]
        
        bin_indices = np.digitize(projections, bin_edges) - 1
        
        bin_indices = np.clip(bin_indices, 0, len(probs) - 1)
        
        sample_probabilities[:, i] = probs[bin_indices]
    
    # Compute mean probability across all projections
    mean_probabilities = sample_probabilities.mean(axis=1)
    
    # Convert to anomaly scores
    anomaly_scores = -np.log(mean_probabilities + 1e-10)
    
    return anomaly_scores

### 2.1.3

In [ ]:
n_test = 500
X_test = np.random.uniform(-3, 3, size=(n_test, 2))

print(X_test.shape)
print(X_test.min(), X_test.max())

# Compute anomaly scores
anomaly_scores = compute_anomaly_scores(X_test, projection_vectors, histograms, probabilities)

print(f"\nAnomaly scores computed:")
print(f"Min score: {anomaly_scores.min():.4f}")
print(f"Max score: {anomaly_scores.max():.4f}")
print(f"Mean score: {anomaly_scores.mean():.4f}")
print(f"Std score: {anomaly_scores.std():.4f}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Plot 1: Training data vs Test data
ax = axes[0]
ax.scatter(X_train[:, 0], X_train[:, 1], alpha=0.4, s=20, c='blue', label='Training (Normal)')
ax.scatter(X_test[:, 0], X_test[:, 1], alpha=0.4, s=20, c='red', label='Test (Uniform)')
ax.set_title('Training vs Test Data Distribution', fontsize=14, fontweight='bold')
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axis('equal')

# Plot 2: Test data colored by anomaly score
ax = axes[1]
scatter = ax.scatter(X_test[:, 0], X_test[:, 1], 
                     c=anomaly_scores, cmap='RdYlBu_r', 
                     s=50, alpha=0.7, edgecolors='black', linewidth=0.5)
plt.colorbar(scatter, ax=ax, label='Anomaly Score')

# Overlay training data distribution (faintly)
ax.scatter(X_train[:, 0], X_train[:, 1], alpha=0.1, s=10, c='blue', marker='.')

ax.set_title(f'Test Data with Anomaly Scores ({n_bins} bins)', fontsize=14, fontweight='bold')
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.grid(True, alpha=0.3)
ax.axis('equal')

plt.tight_layout()
plt.show()

### 2.1.4

In [ ]:
# Test different numbers of bins
bin_values = [5, 10, 20, 30, 50, 100]

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.flatten()

for idx, n_bins_test in enumerate(bin_values):
    # Build histograms with different bin count
    histograms_test, probabilities_test = build_histograms(
        X_train, projection_vectors, n_bins=n_bins_test
    )
    
    # Compute anomaly scores
    anomaly_scores_test = compute_anomaly_scores(
        X_test, projection_vectors, histograms_test, probabilities_test
    )
    
    # Plot
    ax = axes[idx]
    scatter = ax.scatter(X_test[:, 0], X_test[:, 1], 
                        c=anomaly_scores_test, cmap='RdYlBu_r', 
                        s=40, alpha=0.7, edgecolors='black', linewidth=0.3)
    
    # Overlay training data
    ax.scatter(X_train[:, 0], X_train[:, 1], alpha=0.1, s=5, c='blue', marker='.')
    
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label('Anomaly Score', fontsize=9)
    
    ax.set_title(f'{n_bins_test} bins\nScore range: [{anomaly_scores_test.min():.2f}, {anomaly_scores_test.max():.2f}]', 
                fontsize=12, fontweight='bold')
    ax.set_xlabel('Feature 1', fontsize=10)
    ax.set_ylabel('Feature 2', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.axis('equal')

plt.suptitle('Effect of Number of Bins on Anomaly Score Map', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
print("="*70)
print(f"{'Bins':<10} {'Min Score':<12} {'Max Score':<12} {'Mean Score':<12} {'Std Score':<12}")
print("="*70)

for n_bins_test in bin_values:
    histograms_test, probabilities_test = build_histograms(
        X_train, projection_vectors, n_bins=n_bins_test
    )
    anomaly_scores_test = compute_anomaly_scores(
        X_test, projection_vectors, histograms_test, probabilities_test
    )
    
    print(f"{n_bins_test:<10} {anomaly_scores_test.min():<12.4f} "
          f"{anomaly_scores_test.max():<12.4f} {anomaly_scores_test.mean():<12.4f} "
          f"{anomaly_scores_test.std():<12.4f}")


## 2.2

In [ ]:
from pyod.models.iforest import IForest
from pyod.models.loda import LODA

In [ ]:
# Generate 2 clusters with specified centers
X_train_ex2, y_train_ex2 = make_blobs(
    n_samples=[500, 500],
    n_features=2,
    centers=[[10, 0], [0, 10]],  # Two cluster centers
    cluster_std=1.0,  # Standard deviation = 1
    random_state=42
)

print(X_train_ex2.shape)
print(f"Cluster 1 center: [10, 0]")
print(f"Cluster 2 center: [0, 10]")

plt.figure(figsize=(8, 8))
plt.scatter(X_train_ex2[y_train_ex2==0, 0], X_train_ex2[y_train_ex2==0, 1], 
            alpha=0.6, s=30, c='blue', label='Cluster 1 (10, 0)')
plt.scatter(X_train_ex2[y_train_ex2==1, 0], X_train_ex2[y_train_ex2==1, 1], 
            alpha=0.6, s=30, c='green', label='Cluster 2 (0, 10)')
plt.title('Training Data: Two Clusters', fontsize=14, fontweight='bold')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.tight_layout()
plt.show()

### 2.2.2

In [ ]:
iforest_ex2 = IForest(
    n_estimators=100,
    contamination=0.02,
    random_state=42
)

iforest_ex2.fit(X_train_ex2)

n_test_ex2 = 1000
X_test_ex2 = np.random.uniform(-10, 20, size=(n_test_ex2, 2))

print(X_test_ex2.shape)
print(X_test_ex2.min(), X_test_ex2.max())

### 2.2.3

In [ ]:
# Compute anomaly scores for test data
scores_iforest = iforest_ex2.decision_function(X_test_ex2)

print(f"IForest Anomaly Scores:")
print(f"Min: {scores_iforest.min():.4f}")
print(f"Max: {scores_iforest.max():.4f}")
print(f"Mean: {scores_iforest.mean():.4f}")

# colormap
plt.figure(figsize=(10, 10))
scatter = plt.scatter(X_test_ex2[:, 0], X_test_ex2[:, 1], 
                     c=scores_iforest, cmap='RdYlBu_r', 
                     s=30, alpha=0.7, edgecolors='black', linewidth=0.3)

# Overlay training data
plt.scatter(X_train_ex2[:, 0], X_train_ex2[:, 1], 
           alpha=0.2, s=10, c='gray', marker='.')

plt.colorbar(scatter, label='Anomaly Score')
plt.title('IForest: Anomaly Scores', 
         fontsize=14, fontweight='bold')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.tight_layout()
plt.show()

### 2.2.4

In [ ]:
# Train LODA model
loda_ex2 = LODA(contamination=0.02, n_bins=10, n_random_cuts=100)
loda_ex2.fit(X_train_ex2)
scores_loda = loda_ex2.decision_function(X_test_ex2)

In [ ]:
import subprocess
import sys

try:
    import torch
    print("it works")
except ImportError:
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "torch", "--quiet"])
        print("ready")
        
        import torch
        print(f"   Version: {torch.__version__}")
    except Exception as e:
        print("rip")

In [ ]:
from pyod.models.dif import DIF
    
dif_ex2 = DIF(contamination=0.02, hidden_neurons=[64, 32], random_state=42)
dif_ex2.fit(X_train_ex2)
scores_dif = dif_ex2.decision_function(X_test_ex2)

In [ ]:
# Create comparison plots
fig, axes = plt.subplots(1, 3, figsize=(22, 7))
models_data = [
    ('IForest', scores_iforest),
    ('DIF (Deep IForest)', scores_dif),
    ('LODA', scores_loda)
]

for idx, (name, scores) in enumerate(models_data):
    ax = axes[idx]
    
    # Plot test data with scores
    scatter = ax.scatter(X_test_ex2[:, 0], X_test_ex2[:, 1], 
                        c=scores, cmap='RdYlBu_r', 
                        s=30, alpha=0.7, edgecolors='black', linewidth=0.3)
    
    # Overlay training data
    ax.scatter(X_train_ex2[:, 0], X_train_ex2[:, 1], 
              alpha=0.2, s=10, c='gray', marker='.')
    
    plt.colorbar(scatter, ax=ax, label='Anomaly Score')
    ax.set_title(f'{name}\nScore range: [{scores.min():.2f}, {scores.max():.2f}]', 
                fontsize=13, fontweight='bold')
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')
    ax.grid(True, alpha=0.3)
    ax.axis('equal')

plt.suptitle('Comparison: IForest vs DIF vs LODA (2D)', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

### 2.2.5

In [ ]:
bin_configs = [5, 10, 20, 50]

fig, axes = plt.subplots(2, 2, figsize=(16, 14))
axes = axes.flatten()

for idx, n_bins_config in enumerate(bin_configs):
    loda_test = LODA(contamination=0.02, n_bins=n_bins_config, n_random_cuts=100)
    loda_test.fit(X_train_ex2)
    scores_test = loda_test.decision_function(X_test_ex2)
    
    ax = axes[idx]
    scatter = ax.scatter(X_test_ex2[:, 0], X_test_ex2[:, 1], 
                        c=scores_test, cmap='RdYlBu_r', 
                        s=30, alpha=0.7, edgecolors='black', linewidth=0.3)
    
    # Overlay training data
    ax.scatter(X_train_ex2[:, 0], X_train_ex2[:, 1], 
              alpha=0.2, s=10, c='gray', marker='.')
    
    plt.colorbar(scatter, ax=ax, label='Anomaly Score')
    ax.set_title(f'LODA: {n_bins_config} bins\nScore range: [{scores_test.min():.2f}, {scores_test.max():.2f}]', 
                fontsize=12, fontweight='bold')
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')
    ax.grid(True, alpha=0.3)
    ax.axis('equal')

plt.suptitle('LODA: Effect of Number of Bins', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# LODA uses random projections in many directions, averaging over multiple 1D histograms.
# Few bins => coarse boundaries; many bins => finer detail but noisier.
# Unlike IForest, LODA has no axis-parallel artifacts due to random projection directions.

In [ ]:
hidden_configs = [
    [32],          
    [64, 32],      
    [128, 64, 32], 
    [64, 64] 
]

fig, axes = plt.subplots(2, 2, figsize=(16, 14))
axes = axes.flatten()

for idx, hidden_neurons in enumerate(hidden_configs):
    dif_test = DIF(contamination=0.02, hidden_neurons=hidden_neurons, random_state=42)
    dif_test.fit(X_train_ex2)
    scores_test = dif_test.decision_function(X_test_ex2)
    
    ax = axes[idx]
    scatter = ax.scatter(X_test_ex2[:, 0], X_test_ex2[:, 1], 
                        c=scores_test, cmap='RdYlBu_r', 
                        s=30, alpha=0.7, edgecolors='black', linewidth=0.3)
    
    ax.scatter(X_train_ex2[:, 0], X_train_ex2[:, 1], 
              alpha=0.2, s=10, c='gray', marker='.')
    
    plt.colorbar(scatter, ax=ax, label='Anomaly Score')
    ax.set_title(f'DIF: hidden_neurons={hidden_neurons}\nScore range: [{scores_test.min():.2f}, {scores_test.max():.2f}]', 
                fontsize=12, fontweight='bold')
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')
    ax.grid(True, alpha=0.3)
    ax.axis('equal')
    
plt.suptitle('DIF: Effect of Hidden Layer Configuration', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

### 2.2.6

In [ ]:
# Generate 3D training data
X_train_3d, y_train_3d = make_blobs(
    n_samples=[500, 500],
    n_features=3,
    centers=[[0, 10, 0], [10, 0, 10]],
    cluster_std=1.0,
    random_state=42
)

print(X_train_3d.shape)

# Visualize 3D training data
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_train_3d[y_train_3d==0, 0], 
          X_train_3d[y_train_3d==0, 1], 
          X_train_3d[y_train_3d==0, 2],
          alpha=0.6, s=30, c='blue', label='Cluster 1 (0, 10, 0)')
ax.scatter(X_train_3d[y_train_3d==1, 0], 
          X_train_3d[y_train_3d==1, 1], 
          X_train_3d[y_train_3d==1, 2],
          alpha=0.6, s=30, c='green', label='Cluster 2 (10, 0, 10)')

ax.set_title('3D Training Data: Two Clusters', fontsize=14, fontweight='bold')
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_zlabel('Feature 3')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
n_test_3d = 1000
X_test_3d = np.random.uniform(-10, 20, size=(n_test_3d, 3))

print(X_test_3d.shape)

# Train models on 3D data
iforest_3d = IForest(n_estimators=100, contamination=0.02, random_state=42)
iforest_3d.fit(X_train_3d)
scores_iforest_3d = iforest_3d.decision_function(X_test_3d)

loda_3d = LODA(contamination=0.02, n_bins=10, n_random_cuts=100)
loda_3d.fit(X_train_3d)
scores_loda_3d = loda_3d.decision_function(X_test_3d)

In [ ]:
dif_3d = DIF(contamination=0.02, hidden_neurons=[64, 32], random_state=42)
dif_3d.fit(X_train_3d)
scores_dif_3d = dif_3d.decision_function(X_test_3d)

In [ ]:
fig = plt.figure(figsize=(22, 7))
models_3d = [
    ('IForest', scores_iforest_3d),
    ('DIF', scores_dif_3d),
    ('LODA', scores_loda_3d)
]
n_plots = 3

for idx, (name, scores) in enumerate(models_3d):
    ax = fig.add_subplot(1, n_plots, idx+1, projection='3d')
    
    scatter = ax.scatter(X_test_3d[:, 0], X_test_3d[:, 1], X_test_3d[:, 2],
                        c=scores, cmap='RdYlBu_r', 
                        s=20, alpha=0.6, edgecolors='black', linewidth=0.2)
    
    # Overlay training data
    ax.scatter(X_train_3d[:, 0], X_train_3d[:, 1], X_train_3d[:, 2],
              alpha=0.1, s=5, c='gray', marker='.')
    
    plt.colorbar(scatter, ax=ax, label='Anomaly Score', shrink=0.6)
    ax.set_title(f'{name} (3D)\nScore range: [{scores.min():.2f}, {scores.max():.2f}]', 
                fontsize=13, fontweight='bold')
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')
    ax.set_zlabel('Feature 3')

plt.suptitle('3D Comparison: IForest vs DIF vs LODA', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 2.3

### 2.3.1

In [ ]:
import scipy.io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
mat_data = scipy.io.loadmat('shuttle.mat')

# Extract features and labels
X = mat_data['X']
y = mat_data['y'].ravel()  # Flatten to 1D array

print(f"Dataset shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Number of features: {X.shape[1]}")
print(f"Total samples: {X.shape[0]}")
print(f"\n  Normal (0): {(y == 0).sum()} samples ({(y == 0).mean()*100:.2f}%)")
print(f"  Anomaly (1): {(y == 1).sum()} samples ({(y == 1).mean()*100:.2f}%)")

In [ ]:
X_train_shuttle, X_test_shuttle, y_train_shuttle, y_test_shuttle = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)

# Normalize the data
scaler = StandardScaler()
X_train_shuttle_scaled = scaler.fit_transform(X_train_shuttle)
X_test_shuttle_scaled = scaler.transform(X_test_shuttle)

print(f"Training set: {X_train_shuttle_scaled.shape}")
print(f"Test set: {X_test_shuttle_scaled.shape}")
print(f"\nTraining labels distribution:")
print(f"  Normal: {(y_train_shuttle == 0).sum()}")
print(f"  Anomaly: {(y_train_shuttle == 1).sum()}")
print(f"\nTest labels distribution:")
print(f"  Normal: {(y_test_shuttle == 0).sum()}")
print(f"  Anomaly: {(y_test_shuttle == 1).sum()}")

### 2.3.2

In [ ]:
from sklearn.metrics import roc_auc_score, balanced_accuracy_score

# Compute contamination ratio from training data
contamination_ratio = (y_train_shuttle == 1).sum() / len(y_train_shuttle)
print(f"Contamination ratio: {contamination_ratio:.4f}")

print("\n" + "="*70)
print("Single Train-Test Split Evaluation")
print("="*70)

# IForest
iforest_shuttle = IForest(n_estimators=100, contamination=contamination_ratio, random_state=42)
iforest_shuttle.fit(X_train_shuttle_scaled)
y_scores_iforest = iforest_shuttle.decision_function(X_test_shuttle_scaled)
y_pred_iforest = iforest_shuttle.predict(X_test_shuttle_scaled)

ba_iforest = balanced_accuracy_score(y_test_shuttle, y_pred_iforest)
auc_iforest = roc_auc_score(y_test_shuttle, y_scores_iforest)

print(f"\n IForest:")
print(f"   Balanced Accuracy: {ba_iforest:.4f}")
print(f"   ROC AUC: {auc_iforest:.4f}")

# LODA
loda_shuttle = LODA(contamination=contamination_ratio, n_bins=10, n_random_cuts=100)
loda_shuttle.fit(X_train_shuttle_scaled)
y_scores_loda = loda_shuttle.decision_function(X_test_shuttle_scaled)
y_pred_loda = loda_shuttle.predict(X_test_shuttle_scaled)

ba_loda = balanced_accuracy_score(y_test_shuttle, y_pred_loda)
auc_loda = roc_auc_score(y_test_shuttle, y_scores_loda)

print(f"\n LODA:")
print(f"   Balanced Accuracy: {ba_loda:.4f}")
print(f"   ROC AUC: {auc_loda:.4f}")

# DIF
dif_shuttle = DIF(contamination=contamination_ratio, hidden_neurons=[64, 32], random_state=42)
dif_shuttle.fit(X_train_shuttle_scaled)
y_scores_dif = dif_shuttle.decision_function(X_test_shuttle_scaled)
y_pred_dif = dif_shuttle.predict(X_test_shuttle_scaled)

ba_dif = balanced_accuracy_score(y_test_shuttle, y_pred_dif)
auc_dif = roc_auc_score(y_test_shuttle, y_scores_dif)

print(f"\n DIF:")
print(f"   Balanced Accuracy: {ba_dif:.4f}")
print(f"   ROC AUC: {auc_dif:.4f}")

In [ ]:
# Perform 10 different train-test splits and compute mean metrics
# n_splits = 2
n_splits = 10
# random_seeds = [42, 123]
random_seeds = [42, 123, 456, 789, 1011, 1213, 1415, 1617, 1819, 2021]

results = {
    'IForest': {'BA': [], 'AUC': []},
    'LODA': {'BA': [], 'AUC': []},
    'DIF': {'BA': [], 'AUC': []}
}

print("\n" + "="*70)
print(f"Evaluating over {n_splits} different train-test splits...")
print("="*70)

for i, seed in enumerate(random_seeds):
    print(f"\nSplit {i+1}/{n_splits} (seed={seed})...", end=" ")
    
    # Split data
    X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(
        X, y, test_size=0.4, random_state=seed, stratify=y
    )
    
    # Normalize
    scaler_split = StandardScaler()
    X_train_split = scaler_split.fit_transform(X_train_split)
    X_test_split = scaler_split.transform(X_test_split)
    
    # Compute contamination for this split
    contam = (y_train_split == 1).sum() / len(y_train_split)
    
    # IForest
    model_if = IForest(n_estimators=100, contamination=contam, random_state=seed)
    model_if.fit(X_train_split)
    y_pred = model_if.predict(X_test_split)
    y_scores = model_if.decision_function(X_test_split)
    results['IForest']['BA'].append(balanced_accuracy_score(y_test_split, y_pred))
    results['IForest']['AUC'].append(roc_auc_score(y_test_split, y_scores))
    
    # LODA
    model_loda = LODA(contamination=contam, n_bins=10, n_random_cuts=100)
    model_loda.fit(X_train_split)
    y_pred = model_loda.predict(X_test_split)
    y_scores = model_loda.decision_function(X_test_split)
    results['LODA']['BA'].append(balanced_accuracy_score(y_test_split, y_pred))
    results['LODA']['AUC'].append(roc_auc_score(y_test_split, y_scores))
    
    # DIF
    model_dif = DIF(contamination=contam, hidden_neurons=[64, 32], random_state=seed)
    model_dif.fit(X_train_split)
    y_pred = model_dif.predict(X_test_split)
    y_scores = model_dif.decision_function(X_test_split)
    results['DIF']['BA'].append(balanced_accuracy_score(y_test_split, y_pred))
    results['DIF']['AUC'].append(roc_auc_score(y_test_split, y_scores))
    
print("\n" + "="*70)
print("Completed!")
print("="*70)

In [ ]:
# Compute mean and standard deviation for each model
print("\n" + "="*70)
print("RESULTS: Mean ± Std over 10 train-test splits")
print("="*70)

for model_name in ['IForest', 'LODA', 'DIF']:
    ba_values = results[model_name]['BA']
    auc_values = results[model_name]['AUC']
    
    ba_mean = np.mean(ba_values)
    ba_std = np.std(ba_values)
    auc_mean = np.mean(auc_values)
    auc_std = np.std(auc_values)
    
    print(f"\n{model_name}:")
    print(f"   Balanced Accuracy: {ba_mean:.4f} ± {ba_std:.4f}")
    print(f"   ROC AUC:           {auc_mean:.4f} ± {auc_std:.4f}")

print("\n" + "="*70)

In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

models = ['IForest', 'LODA', 'DIF']
colors_bar = ['#3498db', '#e74c3c', '#2ecc71']

# Plot Balanced Accuracy
ax = axes[0]
ba_means = [np.mean(results[m]['BA']) for m in models]
ba_stds = [np.std(results[m]['BA']) for m in models]

bars = ax.bar(models, ba_means, yerr=ba_stds, capsize=8, 
              color=colors_bar, alpha=0.7, edgecolor='black', linewidth=1.5)
ax.set_ylabel('Balanced Accuracy', fontsize=12, fontweight='bold')
ax.set_title('Balanced Accuracy Comparison\n(Mean ± Std over 10 splits)', 
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim([0, 1])

# Add value labels on bars
for bar, mean, std in zip(bars, ba_means, ba_stds):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + std + 0.02,
            f'{mean:.4f}±{std:.4f}',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

# Plot ROC AUC
ax = axes[1]
auc_means = [np.mean(results[m]['AUC']) for m in models]
auc_stds = [np.std(results[m]['AUC']) for m in models]

bars = ax.bar(models, auc_means, yerr=auc_stds, capsize=8,
              color=colors_bar, alpha=0.7, edgecolor='black', linewidth=1.5)
ax.set_ylabel('ROC AUC', fontsize=12, fontweight='bold')
ax.set_title('ROC AUC Comparison\n(Mean ± Std over 10 splits)', 
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim([0, 1])

# Add value labels on bars
for bar, mean, std in zip(bars, auc_means, auc_stds):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + std + 0.02,
            f'{mean:.4f}±{std:.4f}',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.suptitle('Shuttle Dataset: Model Performance Comparison', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()